In [1]:
import sys
import json

In [94]:
hmm_model_file=open("hmmmodel.txt")
hmm_model=json.load(hmm_model_file)
tag_count,emission_count,transition_count,most_probable_count=hmm_model['tag_count'],hmm_model['emission_count'],hmm_model['transition_count'],hmm_model['most_probable_count'][0:5]

In [95]:
test_dev_data_file="hmm-training-data/it_isdt_dev_raw.txt"

In [96]:
test_dev_lines=get_lines_dev_data(test_dev_data_file)

In [108]:
 output = open("hmmoutput.txt", mode='w')

In [111]:
start_tag='<s>'
end_tag='<e>'
for line in test_dev_lines:
    
    word_in_the_corpus_flag,probabilities,backpointers,tags=0,[{}],[{}],[]
    words = line.split(" ")
    for word_index in range(len(words)):
        if(word_index==0):
            first_word=True
        else:
            first_word=False
        if(word_index==len(words)-1):
            last_word=True
        else:
            last_word=False
        if words[word_index] in emission_count.keys():
            tags=emission_count[words[word_index]].keys()
            word_in_the_corpus_flag=1
        else:
            tags=most_probable_count
        if first_word:
            for tag in tags:
                if word_in_the_corpus_flag:
                    probabilities[0][tag]=transition_count[start_tag][tag]*emission_count[words[word_index]][tag]
                    word_in_the_corpus_flag=0
                else:
                    probabilities[0][tag]=transition_count[start_tag][tag]
            backpointers[0][tag]=start_tag
        elif last_word:
            max_probability,max_state=-1 ,''
            probabilities.append({})
            backpointers.append({})
            for tag_j in probabilities[len(words)-1]:
                probabilities_between=transition_count[tag_j]['<e>']*probabilities[len(words)-1][tag_j]
                if max_probability<probabilities_between:
                    max_probability=probabilities_between
                    max_state=tag_j
            probabilities[len(words)-1][end_tag]=max_probability
            backpointers[len(words)-1][end_tag]=max_state
        else:
            probabilities.append({})
            backpointers.append({})
            word_in_the_corpus_flag=0
            if words[word_index] in emission_count.keys():
                tags=emission_count[words[word_index]].keys()
                word_in_the_corpus_flag=1
            else:
                tags=most_probable_count
            for tag in tags:
                if tag==start_tag or tag==end_tag:
                    continue
                max_probability,max_state=-1 ,''
                for tag_j in probabilities[word_index]:
                    if word_in_the_corpus_flag:
                        probabilities_between=transition_count[tag_j][tag]*emission_count[words[word_index]][tag]*probabilities[word_index][tag_j]
                        word_in_the_corpus_flag=0
                    else:
                        probabilities_between=transition_count[start_tag][tag]*probabilities[word_index][tag_j]
                    if max_probability<probabilities_between:
                        max_probability=probabilities_between
                        max_state=tag_j
                probabilities[word_index][end_tag]=max_probability
                backpointers[word_index][end_tag]=max_state
        
    l = len(probabilities)
    
    start = "<e>"
    taggings = words[l-2] + "/" + backpointers[l-1][start]

    start = backpointers[l-1][start]
    probabilities.pop()
    l -= 1
    count=0

    while len(probabilities)-1:
        count+=1
        try:
            backpointers[l-1][start]
        except:
            break
        taggings = words[l-2] + "/" + backpointers[l-1][start] + " "+ taggings
        start = probabilities[l-1][start]
        l -= 1
        probabilities.pop()
            
    output.write(taggings)
    output.write('\n')
        
        
    

In [121]:
model_file = open('hmmmodel.txt')
model = json.load(model_file)

# output file
output = open("hmmoutput.txt", mode='w')

# parameters
emission_prob = model["emission_count"]
transmission_prob = model["transition_count"]
tag_counts = model["tag_count"]
unique_word_tags =model['most_probable_count'][0:5]

input_file_path = "hmm-training-data/it_isdt_dev_raw.txt"
lines = []
with open(input_file_path) as file:
    input_lines = file.readlines()
    for line in input_lines:
        lines.append(line.rstrip())
    
for line in lines:
    words = line.split(" ")
    probabilities = [{}]
    back_pointers = [{}]

    # probability for the first word
    word_exists = 0
    # check if a word is in the dictionary
    if words[0] in emission_prob.keys():
        word_exists = 1
        tags = emission_prob[words[0]].keys()
    else:
        tags = unique_word_tags
    for tag in tags:
        print(transmission_prob['<s>'][tag])
        if word_exists:
            probabilities[0][tag] = transmission_prob['<s>'][tag] * emission_prob[words[0]][tag]
        else:
            probabilities[0][tag] = transmission_prob['<s>'][tag]
        back_pointers[0][tag] = '<s>'

    # probabilities for the rest of the states
    for w in range(1, len(words)):
        word_exists = 0
        if words[w] in emission_prob.keys():
            word_exists = 1
            tags = emission_prob[words[w]].keys()
        else:
            tags = unique_word_tags

        probabilities.append({})
        back_pointers.append({})
        for tag in tags:
            if tag == '<s>' or tag == "<e>":
                continue
            max_prob = - 1
            max_state = ''
            for prev_tag in probabilities[w-1]:
                if word_exists:
                    prob = transmission_prob[prev_tag][tag] * emission_prob[words[w]][tag] * probabilities[w-1][prev_tag]
                else:
                    prob = transmission_prob[prev_tag][tag] * probabilities[w-1][prev_tag]
                if max_prob < prob:
                    max_prob = prob
                    max_state = prev_tag
            probabilities[w][tag] = max_prob
            back_pointers[w][tag] = max_state

    # probability of the end word
    max_prob = - 1
    max_state = ''
    probabilities.append({})
    back_pointers.append({})
    for prev_tag in probabilities[len(words)-1]:
        prob = transmission_prob[prev_tag]['<e>'] * probabilities[len(words)-1][prev_tag]
        if max_prob < prob:
            max_prob = prob
            max_state = prev_tag
    probabilities[len(words)]['<e>'] = max_prob
    back_pointers[len(words)]['<e>'] = max_state

    l = len(probabilities)
    start = "<e>"
    taggings = words[l-2] + "/" + back_pointers[l-1][start]
    start = back_pointers[l-1][start]
    probabilities.pop()
    l -= 1

    while len(probabilities)-1:
        taggings = words[l-2] + "/" + back_pointers[l-1][start] + " "+ taggings
        start = back_pointers[l-1][start]
        l -= 1
        probabilities.pop()

    output.write(taggings)
    output.write('\n')
output.close()

0.0430785594894393
0.20673149977207111
0.002583194043458441
0.0430785594894393
0.000455857772375019
0.028795015955022034
0.0067618902902294485
0.0022033125664792583
0.007445676948791977
0.013371827989667224
0.15932229144506913
0.000455857772375019
0.020285670870688347
0.06906245251481538
0.012612065035708858
0.007445676948791977
0.013371827989667224
0.059337486704148304
0.04034341285518918
0.059337486704148304
0.15932229144506913
0.0430785594894393
0.015727093146938154
0.20673149977207111
0.20673149977207111
0.002583194043458441
0.0430785594894393
0.000455857772375019
0.028795015955022034
0.0067618902902294485
0.20673149977207111
0.0430785594894393
0.020285670870688347
0.000455857772375019
0.059337486704148304
0.0430785594894393
0.059337486704148304
0.06906245251481538
0.0430785594894393
0.007901534721166996
0.028795015955022034
0.15932229144506913
0.20673149977207111
0.020285670870688347
0.0430785594894393
0.012536088740313022
0.04034341285518918
0.059337486704148304
0.206731499772071